In [1]:
from clickhouse_driver import Client
from gensim.models import Word2Vec, Doc2Vec
from urllib import parse
import numpy as np
import pandas as pd
import re
import time
import gensim
from auto_profiling_utils import *
from auto_profiling_model import *
from dti_v3_prep import *

def check_cs(index):
    try:
        client = Client('192.168.0.42', port='9001', send_receive_timeout=int(600000), settings={'max_threads': int(10)})
        client.connection.force_connect()
        if client.connection.connected:
            return client
        else:
            return check_cs(index + 1)
    except:
        return check_cs(index + 1)

def execute_ch(sql, param=None, with_column_types=True):
    client = check_cs(0)
    if client == None:
        sys.exit(1)
    
    result = client.execute(sql, params=param, with_column_types=with_column_types)

    client.disconnect()
    return result

def normal_query(start_date, end_date, limit, interval):
    sql = """
    select
        toStartOfInterval(logtime, INTERVAL {interval}) as lgtime, src_ip, dst_ip,
        arrayStringConcat(groupUniqArray(replaceRegexpAll(replaceRegexpAll(replace(decodeURLComponent(http_host), '/..', ' pathsearcherdetected '), '[\-%./!@#$?,;:&*)(+=0-9_]', ' '), '(\\\\b\\\\w{1}\\\\b)', ' ')), ' ') as host,
        arrayStringConcat(groupUniqArray(replaceRegexpAll(replaceRegexpAll(replace(decodeURLComponent(http_agent), '/..', ' pathsearcherdetected '), '[\-%./!@#$?,;:&*)(+=0-9_]', ' '), '(\\\\b\\\\w{1}\\\\b)', ' ')), ' ') as agent,
        arrayStringConcat(groupUniqArray(replaceRegexpAll(replaceRegexpAll(replace(decodeURLComponent(http_query), '/..', ' pathsearcherdetected '), '[\-%./!@#$?,;:&*)(+=0-9_]', ' '), '(\\\\b\\\\w{1}\\\\b)', ' ')), ' ') as query,
        'normal' as label
    
    from dti.dti_sh_demo_log
    where (logtime >= '{start_date}' and logtime < '{end_date}')
    and hash == 'normal'
    group by lgtime, src_ip, dst_ip
    limit {limit}
    """.replace('{interval}', interval).replace('{start_date}', start_date).replace('{end_date}', end_date).replace('{limit}',limit).replace('{interval}',interval)

    return sql

def attack_query(attack, start_date, end_date, limit, interval):
    sql = """
    select
        toStartOfInterval(logtime, INTERVAL {interval}) as lgtime, src_ip, dst_ip,
        arrayStringConcat(groupUniqArray(replaceRegexpAll(replaceRegexpAll(replace(decodeURLComponent(http_host), '/..', ' pathsearcherdetected '), '[\-%./!@#$?,;:&*)(+=0-9_]', ' '), '(\\\\b\\\\w{1}\\\\b)', ' ')), ' ') as host,
        arrayStringConcat(groupUniqArray(replaceRegexpAll(replaceRegexpAll(replace(decodeURLComponent(http_agent), '/..', ' pathsearcherdetected '), '[\-%./!@#$?,;:&*)(+=0-9_]', ' '), '(\\\\b\\\\w{1}\\\\b)', ' ')), ' ') as agent,
        arrayStringConcat(groupUniqArray(replaceRegexpAll(replaceRegexpAll(replace(decodeURLComponent(http_query), '/..', ' pathsearcherdetected '), '[\-%./!@#$?,;:&*)(+=0-9_]', ' '), '(\\\\b\\\\w{1}\\\\b)', ' ')), ' ') as query,
        '{attack}' as label
    
    from dti.dti_sh_demo_log
    where (logtime >= '{start_date}' and logtime < '{end_date}')
    and hash == '{attack}'
    group by lgtime, src_ip, dst_ip
    limit {limit}
    """.replace('{interval}', interval).replace('{start_date}', start_date).replace('{end_date}', end_date).replace('{limit}',limit).replace('{interval}',interval).replace('{attack}',attack)
    
    return sql

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
config = {
    "common": {
        "model_name": "cnn_model",
        "model_path": "tfidf_model",
        "scaler":"minmaxscaler",
        "vec":"vectorization",
        "encoder":"onehotencoder"
    },
    "train": {
        "data_load": 0,
        "crontab": "*/30 * * * *",
        "now_delta": "minutes=0",
        "prev_delta": "days=30",
        "max_depth":100, ## decision tree depth
        "optimizer_help": ['Adam', 'SGD'],
        "optimizer": 'Adam',
        "learning_rate": 0.0001,
        "batch_size": 32,
        "epochs": 500,
        "result_table": "result"
    },
    "predict": {
        "crontab": "*/1 * * * *",
        "now_delta": "minutes=0",
        "prev_delta": "days=6",
        "batch_size": 8
    }
}

## Data Load

In [3]:
start_time = time.time()

data,meta = execute_ch(normal_query('2021-06-01 00:00:00', '2021-07-01 00:00:00', '10000', '30 minute'))
feats = [m[0] for m in meta]
normal_data = pd.DataFrame(data = data, columns = feats)

attack_data = pd.DataFrame()
for i in ['XSS', 'BEACONING', 'SQL_INJECTION', 'CREDENTIAL']:
    data,meta = execute_ch(attack_query(i, '2021-06-01 00:00:00', '2021-07-01 00:00:00', '10000', '30 minute'))
    feats = [m[0] for m in meta]
    temp_df = pd.DataFrame(data = data, columns = feats)
    attack_data = pd.concat([attack_data, temp_df])
    
data = pd.concat([normal_data, attack_data])
data.reset_index(drop = True, inplace = True)

time.time() - start_time

1.2004621028900146

In [4]:
data['label'].value_counts()

normal           10000
XSS               7926
BEACONING         5349
SQL_INJECTION     4461
CREDENTIAL        4404
Name: label, dtype: int64

## Data Split


In [5]:
# Train/Test data split
y = data[['label']]
x = data.drop('label', axis = 1)

train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.3, random_state=1004)

train_x.reset_index(drop = True, inplace = True)
test_x.reset_index(drop = True, inplace = True)
train_y.reset_index(drop = True, inplace = True)
test_y.reset_index(drop = True, inplace = True)

train_y = pd.get_dummies(train_y['label'])
test_y = pd.get_dummies(test_y['label'])

print(f"No. of training examples: {train_x.shape[0]}")
print(f"No. of testing examples: {test_x.shape[0]}")

save_test_x = test_x.copy()

No. of training examples: 22498
No. of testing examples: 9642


In [6]:
feat_list = ['host', 'agent', 'query']

In [7]:
train_x = train_x[feat_list]
test_x = test_x[feat_list]

## Data Toknization

In [12]:
train_tf = pd.DataFrame(index= train_x.index)

for i in list(train_x):
    train_prep = DataPreprocessing(version = i, mode = 'train', config = config)
    globals()['train_{}'.format(i)] = train_prep.vec_module(train_x[[i]], col_list = i)
    globals()['train_{}'.format(i)].columns = [i +'_'+ j for j in list(globals()['train_{}'.format(i)])]
    train_tf = pd.merge(train_tf, globals()['train_{}'.format(i)], right_index = True, left_index =  True)

In [13]:
test_tf = pd.DataFrame(index= test_x.index)

for i in list(test_x):
    test_prep = DataPreprocessing(version = i, mode = 'test', config = config)
    globals()['test_{}'.format(i)] = test_prep.vec_module(test_x[[i]], col_list = i)
    globals()['test_{}'.format(i)].columns = [i +'_'+ j for j in list(globals()['test_{}'.format(i)])]
    test_tf = pd.merge(test_tf, globals()['test_{}'.format(i)], right_index = True, left_index =  True)

In [14]:
cnn_train_x = np.array(train_tf).reshape(train_tf.shape[0], 1, train_tf.shape[1], 1)
cnn_test_x = np.array(test_tf).reshape(test_tf.shape[0], 1, test_tf.shape[1], 1)

In [15]:
config["x_data_shape"] = cnn_train_x.shape
config["y_data_shape"] = train_y.shape

In [19]:
model = AttackClassification(version='1209', mode='train', config=config)
model.optimize_nn(cnn_train_x, train_y)

Epoch 1/500
704/704 [==============================] - 4s 6ms/step - loss: 0.4604 - categorical_crossentropy: 0.4604 - accuracy: 0.8375
Epoch 2/500
704/704 [==============================] - 4s 6ms/step - loss: 0.1001 - categorical_crossentropy: 0.1001 - accuracy: 0.9696
Epoch 3/500
704/704 [==============================] - 4s 6ms/step - loss: 0.0766 - categorical_crossentropy: 0.0766 - accuracy: 0.9765
Epoch 4/500
704/704 [==============================] - 4s 6ms/step - loss: 0.0707 - categorical_crossentropy: 0.0707 - accuracy: 0.9756
Epoch 5/500
704/704 [==============================] - 4s 6ms/step - loss: 0.0656 - categorical_crossentropy: 0.0656 - accuracy: 0.9783
Epoch 6/500
704/704 [==============================] - 4s 6ms/step - loss: 0.0625 - categorical_crossentropy: 0.0625 - accuracy: 0.9793
Epoch 7/500
704/704 [==============================] - 4s 6ms/step - loss: 0.0594 - categorical_crossentropy: 0.0594 - accuracy: 0.9795
Epoch 00007: early stopping


('MODEL HAS BEEN SAVED TO /home/ctilab/workspace/sophia/NLP/tfidf_model/1209',
 <tensorflow.python.keras.callbacks.History at 0x7f8fe2f07d50>)

In [20]:
true, pred = model.validation(cnn_train_x, train_y.values)

CONFUSION MATRIX
[[3768    0    1    0    2]
 [   1 3068    1    0    6]
 [   0    0 2751    0  374]
 [   0    0    0 5524    2]
 [  12    3   20    0 6965]]
ACCURACY SCORE : 0.9812427771357454


In [21]:
true, pred = model.validation(cnn_test_x, test_y.values)

CONFUSION MATRIX
[[1562    1    7    0    8]
 [   5 1271   14    0   38]
 [   0    0 1179    0  157]
 [   0    0    0 2399    1]
 [   4    6    4    0 2986]]
ACCURACY SCORE : 0.9745903339556109
